In [1]:
import numpy as np 
import math
from itertools import product
import matplotlib.pyplot as plt
plt.style.use('default')

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit.primitives import Sampler, BackendSampler
from qiskit import Aer, QuantumRegister
from qiskit.circuit import QuantumCircuit

from os.path import dirname, abspath
import sys
parent_dir = dirname(dirname(abspath(' ')))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from src.optimizer.Quantum_Optimizer_FGSR_b import *
from src.encode.encode_permutation import *
from src.bitflip_strategy.local_search_general import *

import warnings
warnings.simplefilter('ignore')

In [2]:
np.random.seed(333)

n_loc = 10

X_len = 100
loc_data = np.random.randint(low=1, high=X_len, size=(n_loc, 2))
loc_data = loc_data / X_len

print(
    '-'*50,
    'Problem Type: TSP',
    f'   number of locations: {n_loc}',
    '-'*50,
    sep='\n'
)

--------------------------------------------------
Problem Type: TSP
   number of locations: 10
--------------------------------------------------


In [3]:
D = np.zeros((n_loc, n_loc))

for j, k in product(range(n_loc), range(n_loc)):
    if j==k: continue
    D[j, k] = np.linalg.norm(loc_data[j] - loc_data[k], ord=2)

In [4]:
mdl = QuadraticProgram('TSP')

for i,j in product(range(n_loc), range(n_loc)):
    mdl.binary_var(name=f'x({i},{j})')

cost = {(f'x({k},{i})', f'x({(k-1)%n_loc},{j})'): D[i,j]/2 for i,j,k in product(range(n_loc),range(n_loc),range(n_loc))}
cost.update( {(f'x({k},{i})', f'x({(k+1)%n_loc},{j})'): D[i,j]/2 for i,j,k in product(range(n_loc),range(n_loc),range(n_loc))} )
mdl.minimize(quadratic=cost)

for i in range(n_loc):
    mdl.linear_constraint(linear={f'x({i},{j})':1 for j in range(n_loc)}, sense='==', rhs=1)

for i in range(n_loc):
    mdl.linear_constraint(linear={f'x({j},{i})':1 for j in range(n_loc)}, sense='==', rhs=1)

In [5]:
num_qubits = math.ceil(np.log2(math.factorial(n_loc)))

print(f'number of system qubits: {num_qubits}')

number of system qubits: 22


In [6]:
efficient_encode = create_encode()
efficient_encode.set_encode(n_loc)
decode = efficient_encode.decode

In [7]:
Q_obj = csr_array(mdl.objective.quadratic.to_array())

def objective(x):
    x = decode(x)
    val = x @ Q_obj @ x
    return val

In [8]:
# max number of random flips for each bitstring
n_flips = num_qubits

def search(x):
    x = local_search(x, func=objective, maxiter=n_flips)
    return x

In [9]:
n_reps = 3

q = QuantumRegister(num_qubits, name='q')
sub_qc = QuantumCircuit(q)
qc_params = ParameterVector(name='theta', length=num_qubits*n_reps)

for l in range(n_reps):
    k = 0
    while 2*k < num_qubits-1:
        sub_qc.cz(q[2*k], q[2*k+1])
        k += 1
            
    k = 0
    while 2*k+1 < num_qubits-1:
        sub_qc.cz(q[2*k+1], q[2*k+2])
        k += 1

    for k in range(num_qubits):
        sub_qc.ry(qc_params[l*num_qubits+k], q[k])

    sub_qc.barrier()

In [10]:
shots = 2**10
backend = Aer.get_backend('aer_simulator_matrix_product_state')

sampler = BackendSampler(backend=backend)
sampler.set_options(shots=shots)

print(f'shots = {shots}')

shots = 1024


In [11]:
# configure
alpha = 0.4
max_epoch = 1
restart_s = 0.7
restart_std = 0

In [12]:
cplex_result = CplexOptimizer().solve(mdl)
cplex_optval = cplex_result.fval
refval = cplex_optval

print(refval)

2.6313158388251603


In [13]:
QuantumSolver = QuantumOptimizer()
QuantumSolver.set_optimizer(num_qubits, sub_qc, sampler)

QuantumSolver.run(
    objective = objective,
    search = search,
    alpha = alpha,
    random_update = True,
    max_epoch = max_epoch,
    restart_s = restart_s,
    restart_std = restart_std,
    refval = refval,
    intermediate_result_display = True
    )

epoch = 0 	 step = 0 	 objval = 2.631 	 cvar = 4.235   	 s-prob = 0.000e+00
epoch = 0 	 step = 0 	 objval = 2.631 	 cvar = 2.940   	 s-prob = 3.145e-01
epoch = 0 	 step = 1 	 objval = 2.631 	 cvar = 2.675   	 s-prob = 3.750e-01
epoch = 0 	 step = 2 	 objval = 2.631 	 cvar = 2.631   	 s-prob = 4.629e-01
epoch = 0 	 step = 3 	 objval = 2.631 	 cvar = 2.631   	 s-prob = 4.629e-01
epoch = 0 	 step = 4 	 objval = 2.631 	 cvar = 2.631   	 s-prob = 4.961e-01
epoch = 0 	 step = 5 	 objval = 2.631 	 cvar = 2.631   	 s-prob = 5.723e-01
epoch = 0 	 step = 6 	 objval = 2.631 	 cvar = 2.631   	 s-prob = 6.074e-01
epoch = 0 	 step = 7 	 objval = 2.631 	 cvar = 2.631   	 s-prob = 5.889e-01
epoch = 0 	 step = 8 	 objval = 2.631 	 cvar = 2.631   	 s-prob = 5.967e-01
epoch = 0 	 step = 9 	 objval = 2.631 	 cvar = 2.631   	 s-prob = 5.967e-01
epoch = 0 	 step = 10 	 objval = 2.631 	 cvar = 2.631   	 s-prob = 6.094e-01
epoch = 0 	 step = 11 	 objval = 2.631 	 cvar = 2.631   	 s-prob = 5.879e-01
epoch = 0 

In [14]:
result = QuantumSolver.elog

L1 = [3*i+1 for i in range(len(result))]
L2 = list(result)
L3 = list(QuantumSolver.plog)
L4 = list(QuantumSolver.log)

In [15]:
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['mathtext.default'] = 'it'

fig, ax = plt.subplots(figsize=(10, 4))
ax2 = ax.twinx()

ax.plot(L1, L2, color='navy', linewidth=1, label='Mean (Cost)')
ax.plot(L1, L4, color='darkgreen', linewidth=1, label='Best Sampled Value')

ax.set_ylabel('Cost (CVaR)')
ax.set_xlabel('Eval Count')

ax.hlines(refval, xmin=0, xmax=0, color='darkorange', label='Mean (Probability)')
ax.hlines(refval, xmin=1, xmax=L1[-1], color='grey', linestyle='dashed', linewidth=1, label='Optimal Value')
ax.legend(loc='lower center', bbox_to_anchor=(.5, 1.0), ncol=4)

ax2.plot(L1, L3, color='darkorange', linewidth=1)
ax2.set_ylabel('Probability')
ax2.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])

plt.show()